<div class="div_image pull-right">
    <div class = "image image_topic pull-right">
        <img src = "https://i.imgur.com/EGtMXKh.jpg?1">
    </div>
</div>

# <b>Capstone Project: Choosing the best heroes to climb the rankings</b>


In Part 4, I built a artificial neural network model for our recommender system. In this part, I will go through the process of building the recommender system.


# Import Libraries


In [ ]:
from wtforms.validators import DataRequired
from wtforms import SelectField, SubmitField
from flask_wtf import FlaskForm
from flask import Flask, render_template, session, redirect, url_for
import tensorflow as tf
import numpy as np

import json

import warnings
warnings.filterwarnings('ignore')

# Creating the Recommender System


Our aim for the recommender system is for a person to choose the best available heroes in order to better optimise their team composition to achieve the highest win rate. The greedy algorithm will help in deciding which heroes will be the best option.

<b>Greedy Algorithm</b>

A greedy algorithm is a method of solving a problem that chooses the best solution available at that time. It works on the following approach:

1. It builds the solution piece by piece
2. For each step, it offers the piece offering the most immediate benefit.

In this context, the algorithm will look at the current state of hero selection from both teams and then browse through the list of available candidates and choose the heroes that provide the best win rates.

Finally, I will deploy this on an html page where people can input their choices and the algorithm will return the recommended heroes based on the hero selection.


## Heroes Dictionary


First, I need to create a dictionary of heroes based on their ID. For this, we use the heroes JSON file that we scraped in Part 1 to help us. We will create a few different functions that we will need when building our recommender system:

1. Create a function to return the name of heroes based on the hero_id. As our algorithm will be based on the hero_id, the user will need to know which hero corresponds to which hero_id. I've created a hero_id to localised name dictionary which will help our algorithm translate the hero_id inputs and output the localised name at the frontend.

2. Create a function to return the list of choices for selection. This is similar to the hero_dictionary. However, I have included a 'nil' option as a potential selection so that the algorithm will not register the choice should the input be empty.


In [ ]:
# Import heroes data from json file

heroes_json = 'heroes.json'

# Create


def hero_dictionary(heroes_json):
    with open(heroes_json, 'r') as fp:
        heroes = json.load(fp)

    hero_dict = {heroes[num]['id']: heroes[num]['localized_name']
                 for num in range(0, len(heroes))}

    return hero_dict

# Create hero input choices


def choice(heroes_json):
    with open(heroes_json, 'r') as fp:
        heroes = json.load(fp)

    hero_dict = {heroes[num]['id']: heroes[num]['localized_name']
                 for num in range(0, len(heroes))}

    choices = list(hero_dict.items())
    nil_choice = ('Nil', 'Nil')
    choices.insert(0, nil_choice)
    return choices


## Code Outline

The steps to create the recommender system are as follows:

1. Create a NN predictor class to hold our Neural Network model that will take in the input and output the predicted win rate.
2. Create an engine to wrap around the neural network predictor and run the algorithm


In [ ]:
# Store Variables that may change
FINAL_HERO_ID = hero_ids[-1]
NUM_FEATURES = FINAL_HERO_ID * 2
ENV_PATH = 'recommender/model.h5'
radiant_team = [45, 39, 87, 66]
dire_team = [4, 5, 13, 99, 75]

missing_ids = []
for num in range(0, FINAL_HERO_ID):
    if num not in hero_ids:
        missing_ids.append(num)


In [ ]:
class NNPredictor:
    '''Load Model from the environment path for us to train'''

    def __init__(self, env_path=ENV_PATH):
        self.model = tf.keras.models.load_model(env_path)

    def transform(self, my_team, their_team):
        '''Transform our inputs into the tensorflow input array of shape (1, 246). Slice list of IDs from heroes list '''
        X = np.zeros((1, (NUM_FEATURES+1)))
        '''Slice hero_id chosen onto the array, add number of heroes to hero_id for dire team'''
        for hero_id in my_team:
            X[0][(hero_id)] = 1
        for hero_id in their_team:
            X[0][(hero_id + FINAL_HERO_ID)] = 1

        missing_ids = []
        for num in range(0, FINAL_HERO_ID):
            if num not in hero_ids:
                missing_ids.append(num)

        dire_ids = []
        for id in missing_ids:
            if id > 0:
                dire_id = id + FINAL_HERO_ID
                dire_ids.append(dire_id)

        missing_ids = missing_ids + dire_ids
        '''Delete index number arrays that are missing'''
        X = np.delete(X, missing_ids, axis=1)
        return X

    def recommend(self, my_team, their_team, hero_candidates):
        '''Create two lists - one to hold the current hero selection pair, and one to hold the possible candidate 
        and return the top three heroes with the highest win probability'''
        team_possibilities = [(candidate, my_team + [candidate])
                              for candidate in hero_candidates]

        prob_candidate_pairs = []
        for candidate, team in team_possibilities:
            query = self.transform(team, their_team)
            prob = self.score(query)
            prob_candidate_pairs.append((prob, candidate))
        prob_candidate_pairs = sorted(prob_candidate_pairs, reverse=True)[0:3]
        return prob_candidate_pairs

    def score(self, query):
        '''Score based on our input'''
        rad_prob_1 = self.model.predict(query, verbose=False)[0][0]
        return rad_prob_1

    def predict(self, dream_team, their_team):
        '''Returns the probability of the dream_team winning against their_team.'''
        dream_team_query = self.transform(dream_team, their_team)
        return self.score(dream_team_query)


In [ ]:
class Engine:

    def __init__(self, algorithm):
        self.algorithm = algorithm

    def get_candidates(self, my_team, their_team):
        '''Return a list of hero_IDs to consider for recommending'''
        ids = [
            i for i in hero_ids if i not in my_team and i not in their_team and i not in missing_ids]
        return ids

    def recommend(self, my_team, their_team, human_readable=False):
        '''Return a list of (hero, probability of winning with hero_added) recommend to complete my_team'''

        assert len(my_team) <= 5
        assert len(their_team) <= 5

        hero_candidates = self.get_candidates(my_team, their_team)
        return self.algorithm.recommend(my_team, their_team, hero_candidates)

    def predict(self, dream_team, their_team):
        '''Return the probability of the dream_team winning against their team'''
        return self.algorithm.predict(dream_team, their_team)


# Flask App

Lastly, after building the engine and the predictor, I will create an app using flask and deploy it to the web and deploy it to Heroku. 

To help in the html portion, i have use Flask Wtforms that will help generate our html template. Wtforms is a simple flexible forms validation and rendering library that will help in python html web development. The flask form will be used to create our selection dropdown list for each hero.

The app can be assessed at the following URL:

https://dota-2-hero-recommender.herokuapp.com/

In [ ]:
app = Flask(__name__, template_folder='html_templates')

app.config['SECRET_KEY'] = 'mysecretkey'


class HeroForm(FlaskForm):

    radiant_hero_1 = SelectField(
        u'Your First Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    radiant_hero_2 = SelectField(
        u'Your Second Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    radiant_hero_3 = SelectField(
        u'Your Third Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    radiant_hero_4 = SelectField(
        u'Your Fourth Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    radiant_hero_5 = SelectField(
        u'Your Fifth Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_1 = SelectField(u'Enemy First Hero:',
                              choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_2 = SelectField(
        u'Enemy Second Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_3 = SelectField(u'Enemy Third Hero:',
                              choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_4 = SelectField(
        u'Enemy Fourth Hero:', choices=choice(heroes_json), validators=[DataRequired()])
    dire_hero_5 = SelectField(u'Enemy Fifth Hero:',
                              choices=choice(heroes_json), validators=[DataRequired()])
    submit = SubmitField('Recommend')


@app.route('/', methods=['GET', 'POST'])
def home():
    form = HeroForm()
    if form.validate_on_submit():
        session['radiant_hero_1'] = form.radiant_hero_1.data
        session['radiant_hero_2'] = form.radiant_hero_2.data
        session['radiant_hero_3'] = form.radiant_hero_3.data
        session['radiant_hero_4'] = form.radiant_hero_4.data
        session['radiant_hero_5'] = form.radiant_hero_5.data
        session['dire_hero_1'] = form.dire_hero_1.data
        session['dire_hero_2'] = form.dire_hero_2.data
        session['dire_hero_3'] = form.dire_hero_3.data
        session['dire_hero_4'] = form.dire_hero_4.data
        session['dire_hero_5'] = form.dire_hero_5.data

        return redirect(url_for("recommendation"))

    return render_template('home.html', form=form)


@app.route('/recommendation')
def recommendation():

    radiant_team = (filter(lambda x: x != 'Nil', [session['radiant_hero_1'],
                                                  session['radiant_hero_2'],
                                                  session['radiant_hero_3'],
                                                  session['radiant_hero_4'],
                                                  session['radiant_hero_5']]))
    dire_team = (filter(lambda x: x != 'Nil', [session['dire_hero_1'],
                                               session['dire_hero_2'],
                                               session['dire_hero_3'],
                                               session['dire_hero_4'],
                                               session['dire_hero_5']]))
    radiant_team = [[int(s) for s in sublist] for sublist in radiant_team]
    dire_team = [[int(s) for s in sublist] for sublist in dire_team]

    my_team = radiant_team
    their_team = dire_team

    if len(my_team) >= 5:
        return 'Your Team is Full! Please remove a hero from your team for the system to recommend.'
    elif len(my_team) == 0 and len(my_team) == 0:
        return 'Your Team is Empty! Please fill in at least one hero to continue.'
    else:
        engine = Engine(NNPredictor())
        prob_recommendation_pairs = engine.recommend(my_team, their_team)
        recommendations = [hero for prob,
                           hero in prob_recommendation_pairs]
        heroes = [(hero_dictionary(heroes_json)[hero])
                  for hero in recommendations]
        prob = (round(engine.predict(my_team, their_team) * 100, 2))
        return render_template('recommendation.html', prediction_heroes=f'{heroes}', prediction_wins=f'{prob}')


if __name__ == "__main__":
    app.debug = True
    app.run()


----
# Further Research

1. One of the ways to improve the predictive power of the win rate will be to incorporate more tangible measures such as player's proficiency with the hero, or items chosen.

2. A greedy algorithm search may not be the best algorithm to search for the best hero as it only considers the immediate future to decide the global optimal solution. Hence, the overall optimal solution may hence differ from the solution the algorithm chooses. Further research into other search algorithms could be tested to see if it produces better predictive potential.

3. As Dota 2 goes through many balance changes, which changes the power levels of the heroes, the predictive power hence would not be as accurate for future balance changes. Further scraping of data will need to be done for each balance patch.